# Import 311 CARE/CARE+ Requests

In [8]:
import numpy as np
import pandas as pd
import geopandas as gpd
import intake
from shapely.geometry import Point

In [3]:
catalog = intake.open_catalog('../catalogs/*.yml')

In [4]:
df = pd.read_csv('s3://city-of-los-angeles-data-lake/public-health-dashboard/data/raw/MyLA311_Service_Requests.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.head()

,SRNumber,CreatedDate,UpdatedDate,ActionTaken,Owner,RequestType,Status,RequestSource,CreatedByUserOrganization,MobileOS,...,Location,TBMPage,TBMColumn,TBMRow,APC,CD,CDMember,NC,NCName,PolicePrecinct
0,1-1487677271,11/06/2019 01:17:37 PM,11/06/2019 01:47:38 PM,SR Created,BOS,Homeless Encampment,Pending,Self Service,Self Service,NaN,...,"(34.224884174, -118.465658029)",531.0,H,1.0,North Valley APC,6.0,Nury Martinez,112.0,NORTH HILLS EAST,MISSION
1,1-1487079981,11/05/2019 11:02:38 PM,11/06/2019 09:23:23 PM,SR Created,BOS,Illegal Dumping Pickup,Pending,Call,BOS,NaN,...,"(34.0387039133, -118.253848054)",634.0,E,5.0,Central APC,14.0,Jose Huizar,52.0,DOWNTOWN LOS ANGELES,CENTRAL
2,1-1487079951,11/05/2019 11:01:17 PM,11/06/2019 10:41:56 PM,SR Created,BOS,Bulky Items,Pending,Call,BOS,NaN,...,"(34.0387039133, -118.253848054)",634.0,E,5.0,Central APC,14.0,Jose Huizar,52.0,DOWNTOWN LOS ANGELES,CENTRAL
3,1-1487080711,11/05/2019 10:58:10 PM,11/06/2019 10:41:44 PM,SR Created,BOS,Bulky Items,Pending,Call,BOS,NaN,...,"(34.0387039133, -118.253848054)",634.0,E,5.0,Central APC,14.0,Jose Huizar,52.0,DOWNTOWN LOS ANGELES,CENTRAL
4,1-1487051388,11/05/2019 09:21:10 PM,11/06/2019 10:17:54 PM,SR Created,BOS,Homeless Encampment,Pending,Mobile App,Self Service,iOS,...,"(34.1544585132, -118.39640398)",562.0,G,4.0,South Valley APC,2.0,Paul Krekorian,25.0,NC VALLEY VILLAGE,NORTH HOLLYWOOD


In [6]:
df.columns

Index(['SRNumber', 'CreatedDate', 'UpdatedDate', 'ActionTaken', 'Owner',
       'RequestType', 'Status', 'RequestSource', 'CreatedByUserOrganization',
       'MobileOS', 'Anonymous', 'AssignTo', 'ServiceDate', 'ClosedDate',
       'DateServiceRendered', 'ReasonCode', 'ResolutionCode',
       'AddressVerified', 'ApproximateAddress', 'Address', 'HouseNumber',
       'Direction', 'StreetName', 'Suffix', 'ZipCode', 'Latitude', 'Longitude',
       'Location', 'TBMPage', 'TBMColumn', 'TBMRow', 'APC', 'CD', 'CDMember',
       'NC', 'NCName', 'PolicePrecinct'],
      dtype='object')

In [12]:
df['geometry'] = df.dropna(subset=['Latitude', 'Longitude']).apply(
    lambda x: Point(x.Longitude, x.Latitude), axis = 1)

In [13]:
df.head()

,SRNumber,CreatedDate,UpdatedDate,ActionTaken,Owner,RequestType,Status,RequestSource,CreatedByUserOrganization,MobileOS,...,TBMPage,TBMColumn,TBMRow,APC,CD,CDMember,NC,NCName,PolicePrecinct,geometry
0,1-1487677271,11/06/2019 01:17:37 PM,11/06/2019 01:47:38 PM,SR Created,BOS,Homeless Encampment,Pending,Self Service,Self Service,NaN,...,531.0,H,1.0,North Valley APC,6.0,Nury Martinez,112.0,NORTH HILLS EAST,MISSION,POINT (-118.465658029 34.224884174)
1,1-1487079981,11/05/2019 11:02:38 PM,11/06/2019 09:23:23 PM,SR Created,BOS,Illegal Dumping Pickup,Pending,Call,BOS,NaN,...,634.0,E,5.0,Central APC,14.0,Jose Huizar,52.0,DOWNTOWN LOS ANGELES,CENTRAL,POINT (-118.253848054 34.0387039133)
2,1-1487079951,11/05/2019 11:01:17 PM,11/06/2019 10:41:56 PM,SR Created,BOS,Bulky Items,Pending,Call,BOS,NaN,...,634.0,E,5.0,Central APC,14.0,Jose Huizar,52.0,DOWNTOWN LOS ANGELES,CENTRAL,POINT (-118.253848054 34.0387039133)
3,1-1487080711,11/05/2019 10:58:10 PM,11/06/2019 10:41:44 PM,SR Created,BOS,Bulky Items,Pending,Call,BOS,NaN,...,634.0,E,5.0,Central APC,14.0,Jose Huizar,52.0,DOWNTOWN LOS ANGELES,CENTRAL,POINT (-118.253848054 34.0387039133)
4,1-1487051388,11/05/2019 09:21:10 PM,11/06/2019 10:17:54 PM,SR Created,BOS,Homeless Encampment,Pending,Mobile App,Self Service,iOS,...,562.0,G,4.0,South Valley APC,2.0,Paul Krekorian,25.0,NC VALLEY VILLAGE,NORTH HOLLYWOOD,POINT (-118.39640398 34.15445851320001)


In [7]:
df.ReasonCode.value_counts()

55    91516
75     3172
Name: ReasonCode, dtype: int64

In [14]:
df.RequestType.value_counts()

Homeless Encampment            65535
Illegal Dumping Pickup         16644
Bulky Items                    12071
Service Not Complete             222
Metal/Household Appliances       151
Electronic Waste                  61
Dead Animal Removal                3
Illegal Dumping in Progress        1
Name: RequestType, dtype: int64

In [ ]:
def request_type(row):
    if row.RequestType=='Homeless Encampment'
        return 'homeless'
    elif row.RequestType=='Bulky Items'